In [11]:
import pickle
import random
import time
from typing import List

class Keller4GreedyDescentOptimized:
    DIMENSION = 4
    TARGET_SIZE = 12
    
    def __init__(self, seed=None):
        if seed is None:
            seed = int(time.time())
        random.seed(seed)
        self.seed = seed
        
        filename = f'keller_codes_d{self.DIMENSION}.data'
        with open(filename, 'rb') as f:
            self.codes_int = pickle.load(f)
        
        self.n = len(self.codes_int)
        print(f"K({self.DIMENSION}): {self.n:,} vertices")
    
    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v:
            return False
        if u > v:
            u, v = v, u
        return ((self.codes_int[u] >> v) & 1) == 0
    
    def compute_conflict(self, vertices: List[int]) -> int:
        """Count total conflicts in set."""
        conflicts = 0
        for i in range(len(vertices)):
            for j in range(i + 1, len(vertices)):
                if not self.are_neighbors(vertices[i], vertices[j]):
                    conflicts += 1
        return conflicts
    
    def vertex_conflict(self, v: int, vertices: List[int]) -> int:
        """Count conflicts for vertex v with set."""
        count = 0
        for u in vertices:
            if u != v and not self.are_neighbors(v, u):
                count += 1
        return count
    
    def get_greedy_random_initial_set(self, K: int) -> List[int]:
        """
        Get initial set using greedy-random strategy.
        1. Start with random vertex
        2. Greedily add vertices that minimize conflicts
        3. Some randomness in selection
        """
        # Start with random vertex
        start = random.randint(0, self.n - 1)
        current_set = [start]
        
        # Build set greedily with some randomness
        while len(current_set) < K:
            best_candidates = []
            best_conflicts = K  # Worst possible
            
            # Try random vertices
            candidates = random.sample(range(self.n), min(100, self.n))
            
            for v in candidates:
                if v in current_set:
                    continue
                
                # Calculate conflicts if we add v
                temp_set = current_set + [v]
                conflicts = self.compute_conflict(temp_set)
                
                if conflicts < best_conflicts:
                    best_conflicts = conflicts
                    best_candidates = [v]
                elif conflicts == best_conflicts:
                    best_candidates.append(v)
            
            # Random tie-breaking among best candidates
            if best_candidates:
                chosen = random.choice(best_candidates)
                current_set.append(chosen)
            else:
                # Fallback: add random vertex
                v = random.randint(0, self.n - 1)
                while v in current_set:
                    v = random.randint(0, self.n - 1)
                current_set.append(v)
        
        return current_set
    
    def greedy_descent_min_conflict(self, max_restarts: int = 30) -> List[int]:
        """
        Greedy descent min-conflict with better initial assignment.
        """
        print(f"\nRunning greedy descent min-conflict...")
        
        K = self.TARGET_SIZE
        
        for restart in range(max_restarts):
            # Use greedy-random initial assignment
            C = self.get_greedy_random_initial_set(K)
            current_conflict = self.compute_conflict(C)
            
            if restart == 0:
                print(f"  Initial set (greedy-random): conflict = {current_conflict}")
            
            iteration = 0
            improved = True
            
            # Greedy descent loop
            while improved:
                iteration += 1
                improved = False
                
                if current_conflict == 0:
                    elapsed = time.perf_counter() - self.start_time
                    print(f"   Found {K}-clique in restart {restart}, iteration {iteration}")
                    print(f"    Time: {elapsed:.3f}s")
                    return sorted(C)
                
                # Find best improving swap
                best_improvement = 0
                best_swaps = []
                
                # Check each vertex in current set
                for i, u in enumerate(C):
                    # Check promising candidates (not all 256)
                    candidates = random.sample(range(self.n), min(100, self.n))
                    
                    for v in candidates:
                        if v in C:
                            continue
                        
                        # Try swap
                        new_set = C.copy()
                        new_set[i] = v
                        new_conflict = self.compute_conflict(new_set)
                        improvement = current_conflict - new_conflict
                        
                        if improvement > best_improvement:
                            best_improvement = improvement
                            best_swaps = [(i, v, new_conflict)]
                        elif improvement == best_improvement and improvement > 0:
                            best_swaps.append((i, v, new_conflict))
                
                # Make best move with random tie-breaking
                if best_swaps:
                    i, v, new_conflict = random.choice(best_swaps)
                    C[i] = v
                    current_conflict = new_conflict
                    improved = True
                
                # Stop if taking too long
                if iteration > 50:
                    break
            
            # Progress
            if restart % 5 == 4:
                elapsed = time.perf_counter() - self.start_time
                print(f"  Restart {restart + 1}: conflict = {current_conflict} ({elapsed:.3f}s)")
        
        return []
    
    def fast_search(self, max_attempts: int = 50) -> List[int]:
        """
        Fast search with greedy-random construction.
        """
        print(f"\n searching...")
        
        K = self.TARGET_SIZE
        
        for attempt in range(max_attempts):
            # Try to build a clique greedily with randomness
            clique = self.build_clique_greedy_random(K)
            
            if len(clique) == K and self.compute_conflict(clique) == 0:
                elapsed = time.perf_counter() - self.start_time
                print(f"   Found {K}-clique in attempt {attempt + 1} ({elapsed:.3f}s)")
                return sorted(clique)
        
        return []
    
    def build_clique_greedy_random(self, target_size: int) -> List[int]:
        """Build clique using greedy-random strategy."""
        # Start with random vertex
        start = random.randint(0, self.n - 1)
        clique = [start]
        
        # Candidates that connect to all in clique
        candidates = []
        for v in range(self.n):
            if v != start and self.are_neighbors(start, v):
                candidates.append(v)
        
        random.shuffle(candidates)
        
        while candidates and len(clique) < target_size:
            # Score candidates by connections to other candidates
            scores = []
            for v in candidates:
                score = 0
                for u in candidates:
                    if u != v and self.are_neighbors(v, u):
                        score += 1
                scores.append(score)
            
            # Pick from top candidates
            max_score = max(scores)
            top_indices = [i for i, score in enumerate(scores) if score == max_score]
            chosen_idx = random.choice(top_indices)
            chosen = candidates[chosen_idx]
            
            # Add to clique
            clique.append(chosen)
            
            # Update candidates
            new_candidates = []
            for i, v in enumerate(candidates):
                if i != chosen_idx and self.are_neighbors(chosen, v):
                    new_candidates.append(v)
            
            candidates = new_candidates
            random.shuffle(candidates)
        
        # If we couldn't reach target size, fill with random vertices
        while len(clique) < target_size:
            v = random.randint(0, self.n - 1)
            if v not in clique:
                clique.append(v)
        
        return clique
    
    def verify_clique(self, vertices: List[int]) -> bool:
        return self.compute_conflict(vertices) == 0
    
    def find_clique(self) -> List[int]:
        """Main search function."""
       
        
        self.start_time = time.perf_counter()
        
        # Try fast search first
        #print("\n Running fast greedy-random search...")
        clique = self.fast_search(max_attempts=30)
        
        # If not found, try greedy descent
        if not clique:
            #rint("\n[2] Fast search failed, trying greedy descent...")
            clique = self.greedy_descent_min_conflict(max_restarts=20)
        
        elapsed = time.perf_counter() - self.start_time
        
        # Results
        print(f"\n RESULTS:")
        print(f"  Total time: {elapsed:.3f} seconds")
        
        if clique:
            is_valid = self.verify_clique(clique)
            print(f"  Clique size: {len(clique)}")
            print(f"  Vertices: {sorted(clique)}")
            print(f"  Valid: {is_valid}")
            
            if is_valid:
                print(f"\n SUCCESS: Valid {len(clique)}-clique found")
                
                # Show coordinates
                print(f"\n VERTEX COORDINATES:")
                for v in sorted(clique):
                    coords = [(v >> (2*i)) & 0b11 for i in range(4)]
                    print(f"  {v:3d}: {coords}")
                
                
        else:
            print(f"FAILD NO {self.TARGET_SIZE}-clique FOUND")
        
        return clique

def main():
    """Main execution."""
    print("="*70)
    print("KELLER K(4) - GREEDY DESCENT MIN-CONFLICT")
    print("BY BOUNEB ZINE EL ABIDINE")
    print("="*70)
    
    solver = Keller4GreedyDescentOptimized()
    clique = solver.find_clique()
    
    print("\n" + "="*70)
    print("SEARCH COMPLETE")
    print("="*70)
    
    return clique

if __name__ == "__main__":
    clique = main()

KELLER K(4) - GREEDY DESCENT MIN-CONFLICT
BY BOUNEB ZINE EL ABIDINE
K(4): 256 vertices

 searching...
   Found 12-clique in attempt 1 (0.046s)

 RESULTS:
  Total time: 0.046 seconds
  Clique size: 12
  Vertices: [0, 36, 47, 66, 75, 102, 168, 174, 176, 200, 206, 214]
  Valid: True

 SUCCESS: Valid 12-clique found

 VERTEX COORDINATES:
    0: [0, 0, 0, 0]
   36: [0, 1, 2, 0]
   47: [3, 3, 2, 0]
   66: [2, 0, 0, 1]
   75: [3, 2, 0, 1]
  102: [2, 1, 2, 1]
  168: [0, 2, 2, 2]
  174: [2, 3, 2, 2]
  176: [0, 0, 3, 2]
  200: [0, 2, 0, 3]
  206: [2, 3, 0, 3]
  214: [2, 1, 1, 3]

SEARCH COMPLETE
